# BoxBunny All‑in‑One Test Notebook
This guide walks through camera, IMU, LLM, and ROS smoke tests.
Each section is independent; run only what you need.


## 0) Environment check
Make sure ROS 2 Humble is sourced in this kernel.

In [ ]:
import os
print('ROS_DISTRO:', os.environ.get('ROS_DISTRO', 'not sourced'))


## 1) Camera test (RealSense D435i)
Opens a preview window. Press **q** to quit.

In [ ]:
import pyrealsense2 as rs
import cv2
import numpy as np

pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
pipeline.start(config)

try:
    for _ in range(200):
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not color_frame:
            continue
        img = np.asanyarray(color_frame.get_data())
        cv2.imshow('D435i', img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    pipeline.stop()
    cv2.destroyAllWindows()


## 2) IMU test (MPU6050)
Run IMU node first (see ros_commands.ipynb).

In [ ]:
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Imu

class ImuReader(Node):
    def __init__(self):
        super().__init__('imu_reader')
        self.create_subscription(Imu, 'imu/data', self.cb, 10)
    def cb(self, msg):
        self.get_logger().info(f'ax={msg.linear_acceleration.x:.2f} ay={msg.linear_acceleration.y:.2f} az={msg.linear_acceleration.z:.2f}')

rclpy.init()
node = ImuReader()
try:
    rclpy.spin(node)
except KeyboardInterrupt:
    pass
node.destroy_node()
rclpy.shutdown()


## 3) LLM local test
Loads the GGUF model directly using llama.cpp.

In [ ]:
from llama_cpp import Llama

model_path = '/home/boxbunny/Desktop/doomsday_integration/boxbunny_ws/models/llm/qwen2.5-3b-instruct-q4_k_m.gguf'
llm = Llama(model_path=model_path, n_ctx=512, n_threads=6)

prompt = 'You are a playful boxing coach. Say one short sentence to hype me up.'
result = llm(prompt, max_tokens=32, temperature=0.7, stop=['
'])
print(result['choices'][0]['text'].strip())


## 4) LLM Chat GUI (tuning + prompt editing)
Use this to adjust system prompt, Singlish mode, and speed settings.


In [ ]:
!source /opt/ros/humble/setup.bash && \
  cd /home/boxbunny/Desktop/doomsday_integration/boxbunny_ws && \
  colcon build --symlink-install --packages-select boxbunny_llm && \
  source install/setup.bash && \
  ros2 run boxbunny_llm llm_chat_gui


## 5) ROS topic smoke test
Run the full system first, then check topics below.

In [ ]:
import rclpy
from rclpy.node import Node
from boxbunny_msgs.msg import GloveDetections, PunchEvent

class Smoke(Node):
    def __init__(self):
        super().__init__('boxbunny_smoke')
        self.create_subscription(GloveDetections, 'glove_detections', self.on_glove, 10)
        self.create_subscription(PunchEvent, 'punch_events', self.on_punch, 10)
    def on_glove(self, msg):
        self.get_logger().info(f'gloves={len(msg.detections)}')
    def on_punch(self, msg):
        self.get_logger().info(f'punch={msg.glove} type={msg.punch_type}')

rclpy.init()
node = Smoke()
try:
    rclpy.spin(node)
except KeyboardInterrupt:
    pass
node.destroy_node()
rclpy.shutdown()
